# 5
- 오늘 할 일
    - 데이터를 뽑기 (기존 데이터 보기 + 데이터 뽑는 과정 보고 추가할 rule 뽑기)
    - 좋은 feature를 많이 뽑기
    - 관련 논문들 읽어보기 + 좋은 feature 많이 뽑기

In [1]:
import pickle
from collections import namedtuple
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import pandas as pd
import sys
sys.path.append("/home/angrypark/korean-text-matching-tf/")

from utils.utils import JamoProcessor
from text.tokenizers import SentencePieceTokenizer

Config = namedtuple("config", ["sent_piece_model"])
config = Config("/media/scatter/scatterdisk/tokenizer/sent_piece.100K.model")
processor = JamoProcessor()
tokenizer = SentencePieceTokenizer(config)

def my_word_tokenizer(raw, pos=["Noun", "Alpha", "Verb", "Number"], stopword=[]):
    return [word for word in tokenizer.tokenize(raw)]

def my_char_tokenizer(raw, pos=["Noun", "Alpha", "Verb", "Number"], stopword=[]):
    return [processor.word_to_jamo(word) for word in tokenizer.tokenize(raw)]

---
지난 xgboost, catboost가 중요하게 본 feature는 뭐지?

In [2]:
xgb = pickle.load(open("../models/xgb.pkl", "rb"))
cb = pickle.load(open("../models/cb.pkl", "rb"))

In [3]:
from feature_extractor import FeatureExtractor

In [4]:
feature_extractor = FeatureExtractor()

Pre-trained embedding loaded. Number of OOV : 5272 / 90000


/home/angrypark/angryenv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Restoring parameters from /media/scatter/scatterdisk/reply_matching_model/runs/delstm_1024_nsrandom4_lr1e-3/best_loss/best_loss.ckpt


In [14]:
with open("../data/my_data/test.txt", "r") as f:
    test_A, test_B, test_labels = zip(*[line.split("\t") for line in f])
    test_labels = [1 if l.strip()=="1" else 0 for l in test_labels]

In [7]:
%%time
test_set = feature_extractor.extract_features(test_A, test_B)

CPU times: user 52.6 s, sys: 2.1 s, total: 54.7 s
Wall time: 49.9 s


In [26]:
with open("../data/my_data/train.txt", "r") as f:
    train_A, train_B, train_labels = zip(*[line.strip().split("\t") for line in f])
    train_labels = [1 if l=="1" else 0 for l in train_labels]

In [28]:
%%time
train_set = feature_extractor.extract_features(train_A, train_B)

CPU times: user 19min 55s, sys: 46.6 s, total: 20min 42s
Wall time: 18min 48s


In [29]:
pd.DataFrame(train_set).to_csv("../data/my_data/train_set.csv", index=False)

In [30]:
with open("../data/my_data/val.txt", "r") as f:
    val_A, val_B, val_labels = zip(*[line.strip().split("\t") for line in f])
    val_labels = [1 if l=="1" else 0 for l in val_labels]

In [31]:
%%time
val_set = feature_extractor.extract_features(val_A, val_B)

CPU times: user 1min 3s, sys: 0 ns, total: 1min 3s
Wall time: 58 s


In [32]:
pd.DataFrame(val_set).to_csv("../data/my_data/val_set.csv", index=False)

In [33]:
train_set = pd.DataFrame(train_set)

In [34]:
from xgboost import XGBClassifier
xgb = XGBClassifier()

In [36]:
xgb.fit(train_set, train_labels)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [38]:
xgb.score(pd.DataFrame(test_set), test_labels)

/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7654986522911051

In [39]:
sum(test_labels)

8276

In [40]:
len(test_labels)

19292

In [41]:
sum(train_labels)

76486

In [43]:
len(train_labels)

400000

In [44]:
76486/8276*19292

178294.81778637023

In [45]:
import pickle

In [46]:
pickle.dump(xgb, open("../models/new_xgb.pkl", "wb"))

In [47]:
xgb.score(pd.DataFrame(train_set), train_labels)

/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.887285